In [18]:
import json
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def load_webtoon_info(file_path='webtoon_info.json'):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def scrape_webtoon_comments(title, episode):
    webtoon_info = load_webtoon_info()
    if title not in webtoon_info:
        print(f"웹툰 '{title}' 정보를 찾을 수 없습니다.")
        return
    
    webtoon_id = webtoon_info[title]['id']
    url = f"https://comic.naver.com/webtoon/detail?titleId={webtoon_id}&no={episode}"
    
    headers = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("페이지 요청 실패")
        return
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 브라우저를 띄우지 않고 실행
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(3)
    
    # 관심웹툰 등록자수
    interest_count = driver.find_element(By.CLASS_NAME, 'UserAction__count--jk3vo').text
    
    # 좋아요 수
    like_count = driver.find_element(By.CLASS_NAME, 'u_cnt._count').text
    
    # 별점
    rating = driver.find_element(By.CLASS_NAME, 'UserAction__score--sP1ha').text
    
    # 전체 댓글 보기 버튼 클릭
    try:
        view_comments_btn = driver.find_element(By.CLASS_NAME, 'u_cbox_btn_view_comment')
        view_comments_btn.click()
        time.sleep(2)
    except:
        print("전체 댓글 보기 버튼이 없습니다.")
    
    # 더보기 버튼 클릭 (끝까지)
    while True:
        try:
            more_btn = driver.find_element(By.CLASS_NAME, 'u_cbox_paginate')
            more_btn.click()
            time.sleep(1)
        except:
            break
    
    comments_data = []
    comments = driver.find_elements(By.CLASS_NAME, 'u_cbox_comment_box')
    for comment in comments:
        try:
            text = comment.find_element(By.CLASS_NAME, 'u_cbox_contents').text
            recomm = comment.find_element(By.CLASS_NAME, 'u_cbox_cnt_recomm').text
            unrecomm = comment.find_element(By.CLASS_NAME, 'u_cbox_cnt_unrecomm').text
            comments_data.append({
                "text": text,
                "recomm": recomm,
                "unrecomm": unrecomm
            })
        except NoSuchElementException:
            continue
    
    driver.quit()
    
    # JSON 파일로 저장
    save_path = f"webtoon_data/{title}_{episode}.json"
    os.makedirs('webtoon_data', exist_ok=True)
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump({
            "webtoon": title,
            "episode": episode,
            "interest_count": interest_count,
            "like_count": like_count,
            "rating": rating,
            "comments": comments_data
        }, f, ensure_ascii=False, indent=4)
    
    print(f"{len(comments_data)}개의 댓글이 {save_path}에 저장되었습니다.")

# 예제 실행
# scrape_webtoon_comments('웹툰제목1', 169)

In [19]:
load_webtoon_info()
scrape_webtoon_comments('퀘스트지상주의', 154)

1317개의 댓글이 webtoon_data/퀘스트지상주의_154.json에 저장되었습니다.
